In [2]:
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
import sys
sys.path.append("..")
from Vocabulary import *
import math
import numpy as np

# Next block contains all parameters, which need to be set

In [3]:

has_tag_Rep = True
embedding_path = r"D:\\tagged\\"
embedding_name = 'treeTagged_150w'
outlocation = "D:\\"
New_emb_Name_path = outlocation+embedding_name + '_combined'#later added with String _strategyX
unfiltered_vocab = Vocabulary()
unfiltered_vocab.load(r"C:\Users\weso\Desktop\gauntlet\vocabs\unfilteredvocab_treebank")
dimensions = 500

In [4]:
import numpy as np
import tensorflow as tf
id_dict = {}
word_dict = {}

matrix = []
full_path = os.path.join(embedding_path,embedding_name)
with open(full_path, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

309715


In [5]:
#find all variants of a word, without the tag rep
def group_all_words(id_dict):
    groups = {}
    for word in id_dict:
        #print(word)
        if is_tagged_word(word):
            #print('is tagged')
            baseword = word.split(chr(4))[0]
            #print(baseword)
            if baseword in groups:
                groups[baseword].append(word)
            else:
                groups[baseword] = [word]
        elif word.startswith(chr(4)):#exclude tags
            pass
        else:
            if word in groups:
                groups[word].append(word)
            else:
                groups[word] = [word]
    return groups

def is_tagged_word(word):
    return not word.startswith(chr(4)) and str(chr(4)) in word

groups = group_all_words(id_dict)
print(len(groups))
print(groups['duck'])
print(groups['the'])
print(groups['bank'])
print('---------')
print(str(groups)[:100])
print(groups["splashdown"])

243605
['duck\x04vb', 'duck\x04nnp', 'duck\x04nn']
['the\x04nnp', 'the\x04nfp', 'the\x04dt', 'the\x04rb', 'the\x04']
['bank\x04vb', 'bank\x04jj', 'bank\x04nnp', 'bank\x04nn']
---------
{'tbr': ['tbr\x04nnp'], 'khamhaeng': ['khamhaeng\x04nnp'], 'buffy': ['buffy\x04jj', 'buffy\x04nnp', 
['splashdown\x04nnp', 'splashdown\x04nn']


# helper

In [8]:
def get_count(word):
    return unfiltered_vocab.word_frequency[word]

In [9]:
def save_new_embedding(embedding,path):
    with open(path,'w+',encoding='utf8') as file:
        for id,word in enumerate(embedding):
            if(id % 1000 == 0):
                print(id,len(embedding))
            
            file.write(word)
            vector = embedding[word]
            for coord in vector:
                file.write(' '+str(coord))
            file.write('\n')

# Strategy 1 weighted Sum

In [10]:
#requires tag rep
def mult_strat_weighted(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:

        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if chr(4) in word:
                if(word.startswith(chr(4))):#no rep for tags
                    continue
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                #id_tag  = id_dict[chr(4)+tag]
                #emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
                emb_sum += (get_count(word)/base_count) * (matrix_copy[id_word])
            else:#only in mixed embedding add the base unscaled
                emb_sum += (get_count(word)/base_count) * matrix_copy[id_dict[word]]
        emb[baseword] = emb_sum
    return emb
    

In [11]:
emb1 = mult_strat_weighted(groups,False)
save_new_embedding(emb1,New_emb_Name_path+"_Strat1")

0 243604
1000 243604
2000 243604
3000 243604
4000 243604
5000 243604
6000 243604
7000 243604
8000 243604
9000 243604
10000 243604
11000 243604
12000 243604
13000 243604
14000 243604
15000 243604
16000 243604
17000 243604
18000 243604
19000 243604
20000 243604
21000 243604
22000 243604
23000 243604
24000 243604
25000 243604
26000 243604
27000 243604
28000 243604
29000 243604
30000 243604
31000 243604
32000 243604
33000 243604
34000 243604
35000 243604
36000 243604
37000 243604
38000 243604
39000 243604
40000 243604
41000 243604
42000 243604
43000 243604
44000 243604
45000 243604
46000 243604
47000 243604
48000 243604
49000 243604
50000 243604
51000 243604
52000 243604
53000 243604
54000 243604
55000 243604
56000 243604
57000 243604
58000 243604
59000 243604
60000 243604
61000 243604
62000 243604
63000 243604
64000 243604
65000 243604
66000 243604
67000 243604
68000 243604
69000 243604
70000 243604
71000 243604
72000 243604
73000 243604
74000 243604
75000 243604
76000 243604
77000 243604

# Strategy 2 MultTagRep

In [92]:
#requires tag rep
def mult_tag_rep_strat(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:
        
        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            if chr(4) in word:
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                #id_tag  = id_dict[chr(4)+tag]
                #emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
                emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
            else:#only in mixed embedding add the base unscaled
                emb_sum += (get_count(word)/base_count) * matrix_copy[id_dict[word]]
        emb[baseword] = emb_sum
    return emb

In [93]:
emb2 = mult_tag_rep_strat(groups,False)
save_new_embedding(emb2,New_emb_Name_base+"_Strat2")

[ 0.83655888  0.56230641  1.31466868  0.75338636  0.88259477  0.54048636
  0.66813626  0.24701116  1.08327434  0.73842778  0.93343065  0.58456334
  0.7408845   0.62628036  1.04259515  0.80160135  1.280772    0.67505019
  1.02274889  0.8647537   0.2768535   0.78671927 -0.07325811  0.89054524
  0.79636452  0.9597792   0.72155377  0.25776857  0.59434872  0.45070298
  0.82455262  0.7198299   1.2681247   1.09165162  0.56565395  0.54518698
  0.85127129  0.82685436  1.01986472  0.82717568  0.81580072  0.25343165
  0.71523798  0.80283276  0.66233341  0.41603753  1.03093484  0.8292028
  0.92813965  1.39479177  0.80875336  0.15275364  0.7153622   0.42662276
  0.46856623  0.48922916  0.12875231  1.09962834 -0.2594594   0.74718473
  0.55902573  0.87082431  0.79948158  0.77297602  0.08179791  0.31714618
  1.03793356  0.79005332  0.84827023  0.79805702  0.82308105  0.77717281
  0.73482255  0.90558642  0.36310445  0.22822998  1.20710833  0.77715428
  0.22117303  0.68976168  1.01707578  0.20818995  0.

# Strategy 3 AddTagRep

In [94]:
#requires tag rep
def add_tag_rep_strat(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:
        
        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            if chr(4) in word:
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                #id_tag  = id_dict[chr(4)+tag]
                #emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
                emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] + matrix_copy[id_tag])
            else:#only in mixed embedding add the base unscaled
                emb_sum += matrix_copy[id_dict[word]] * (get_count(word)/base_count)
        emb[baseword] = emb_sum
    return emb

In [95]:
emb3 = add_tag_rep_strat(groups,False)
save_new_embedding(emb3,New_emb_Name_base+"_Strat3")

[ 2.76229148  2.51590141 -0.0420942   3.04173901  2.64908255  1.87738493
  1.45163523  1.19443803  0.6250927   1.69642911  1.61737342  1.52952278
  2.29894397  2.15844273 -0.14075882  1.39544763  2.77200947  3.19888065
  3.13699791  1.70949071  2.79992501  1.41123817  0.95375259  3.11368731
  2.20464367  1.77384376  2.96637402  0.31623668  0.7891416   2.83012274
  2.97160488  1.35648132  2.75739571  2.45551568  1.0200732   2.98060491
  2.04349855  1.16320939  3.09564091  2.30113335  3.00935562  3.09190175
  2.54307032  0.79285289  1.93104478  3.39580113  2.69059502  0.85931519
  2.59802213  1.30132434  2.67013605  0.74340426  2.07360825  2.54885857
  2.48511375  3.35046838  1.12360115  3.55643519 -0.87518359  2.99375467
  1.97027142  2.84140825  1.9278737   2.09939259  1.63945015  2.47593054
  1.59633541  2.1428813   3.37755092  2.96957291  2.53867023  2.52024722
  2.08452366  2.93738024  1.58296975  1.72754244  4.53467243  1.60923323
  2.46810958  1.7468284   2.7909703   1.55511231  1